<a href="https://colab.research.google.com/github/byteLm/or/blob/main/PO_Solucao_AP1_MixProducao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solução de Aula Prática I
> Pesquisa Operacional - Disciplina lecionada na Universidade Federal da Paraíba pelo prof. Teobaldo Bulhões



# Aula prática: Mix de Produção
<sup>Adaptado dos exercícios 2.3 e 2.5 do livro `Pesquisa Operacional`, de `Arenales, Armentano, Morabito e Yanasse`.</sup>

## Exercício 1

### Descrição do problema
Uma fundição tem de produzir 10 toneladas de um tipo de liga metálica e, para isso, tem disponível: lingotes de ferro, grafite e sucata. Dois componentes são relevantes para a liga: carbono e silício. As tabelas a seguir fornecem a fração, em termos percentuais, desses elementos nos ingredientes disponíveis, seus custos unitários, bem como a composição da liga (isto é, porcentagens mínima e máxima de cada componente da liga).

Frações dos elementos (%) nos ingredientes e custo dos ingredientes (R$/ton):

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Carbono | 0.5 | 90 | 9 |
| Silício | 14 | - | 27 |
| Custo | 90 | 180 | 25 |

Frações (%) mínima e máxima dos componentes na liga:

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.0 | 9.5 |
|Silício | 19 | 20 |


Escreva um modelo de otimização linear para determinar as quantidades dos ingredientes para compor a liga metálica, de modo que as especificações técnicas sejam satisfeitas e o custo seja mínimo.

### Resolução

In [1]:
# instalação e importação do pacote mip
!pip install mip

from mip import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.16.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


Carrega Dados

In [2]:
# composição de cada ingrediente
a = {
    'l': {'c': 0.005, 's': 0.14},
    'g': {'c': 0.9,   's': 0.0},
    's': {'c': 0.09,  's': 0.27},
}

# custo
c = {'l': 90, 'g': 180, 's': 25}

# composições mínimas e máximas dos componentes
n = {'c': 0.0, 's': 0.19}  # min
m = {'c': 0.095, 's': 0.2} # max

# quantidade desejada da liga
Q = 10

Cria modelo

In [3]:
modelo = Model(sense=MINIMIZE, solver_name=CBC)

# Criação da variável
x = []
x.append(modelo.add_var(var_type=CONTINUOUS, name=f"x_lingotes", lb=0.0))
x.append(modelo.add_var(var_type=CONTINUOUS, name=f"x_grafite", lb=0.0))
x.append(modelo.add_var(var_type=CONTINUOUS, name=f"x_sucata", lb=0.0))


# Função objetivo = minimizar custo total
modelo.objective = c['l']*x[0] + c['g']*x[1] + c['s']*x[2] # Minimizar esse custo, dado que lingote é x$, grafite é y$ e sucate é z$

# Restrição: quantidade minína de carbono
modelo += a['l']['c']*x[0] + a['g']['c']*x[1] + a['s']['c']*x[2] >= n['c']*10

# Restrição: quantidade máxima de carbono
modelo += a['l']['c']*x[0] + a['g']['c']*x[1] + a['s']['c']*x[2] <= m['c']*10 # Restrição 1: quantidade máxima de carbono

# Restrição: quantidade miníma de silício
modelo += a['l']['s']*x[0] + a['g']['s']*x[1] + a['s']['s']*x[2] >= n['s']*10 # Restrição 2: quantidade minína de silicio
modelo += a['l']['s']*x[0] + a['g']['s']*x[1] + a['s']['s']*x[2] <= m['s']*10 # Restrição 2: quantidade minína de silicio

# a soma total tem que dar 10 toneladas
modelo += x[0] + x[1] + x[2] == 10

modelo.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_lingotes + 180 x_grafite + 25 x_sucata
Subject To
constr(0):  0.00500 x_lingotes + 0.90000 x_grafite + 0.09000 x_sucata >= -0
constr(1):  0.00500 x_lingotes + 0.90000 x_grafite + 0.09000 x_sucata <= 0.95000
constr(2):  0.14000 x_lingotes + 0.27000 x_sucata >= 1.90000
constr(3):  0.14000 x_lingotes + 0.27000 x_sucata <= 2
constr(4):  x_lingotes + x_grafite + x_sucata = 10
Bounds
End



Executa

In [4]:
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")

solve(modelo)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 600.00

Solution:
x_lingotes = 5.38
x_grafite = 0.00
x_sucata = 4.62


## Exercício 2

Agora considere que os ingredientes tem o estoque limitado, de acordo com a tabela abaixo.

| | Lingotes | Grafite | Sucata |
|:---|:---:|:---:|:---:|
| Estoque (ton) | 5 | 5 | 12 |

Como o modelo pode ser modificado para atender a esse requisito?

### Código

Carrega Dados

In [5]:
# estoque
e = {'l': 5, 'g': 5, 's': 12}

Cria modelo

In [6]:
# Apenas precisamos considerar que a quantidade de lingotes, grafite e sucata precisam ser menores que o seu respectivo estoque
modelo += x[0] <= e['l']
modelo += x[1] <= e['g']
modelo += x[1] <= e['s']

modelo.write("modelo2.lp") # salva modelo em arquivo
with open("modelo2.lp") as f: # Lê e exibe conteúdo do arquivo
  print(f.read())

\Problem name: 

Minimize
OBJROW: 90 x_lingotes + 180 x_grafite + 25 x_sucata
Subject To
constr(0):  0.00500 x_lingotes + 0.90000 x_grafite + 0.09000 x_sucata >= -0
constr(1):  0.00500 x_lingotes + 0.90000 x_grafite + 0.09000 x_sucata <= 0.95000
constr(2):  0.14000 x_lingotes + 0.27000 x_sucata >= 1.90000
constr(3):  0.14000 x_lingotes + 0.27000 x_sucata <= 2
constr(4):  x_lingotes + x_grafite + x_sucata = 10
constr(5):  x_lingotes <= 5
constr(6):  x_grafite <= 5
constr(7):  x_grafite <= 12
Bounds
End



Executa

In [7]:
solve(modelo)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 603.70

Solution:
x_lingotes = 5.00
x_grafite = 0.19
x_sucata = 4.81


## Exercício 3

Suponha agora que duas ligas metálicas devem ser preparadas e os mesmos ingredientes são utilizados em ambas. A liga especificada no Exercício 1 é referida como liga 1 e devem ser produzidas 10 toneladas desta liga. Da outra liga, referida como liga 2, devem ser produzidas 6 toneladas e suas composições mínima e máxima são dadas na tabela abaixo.

| | min | max |
|:---|:---:|:---:|
|Carbono | 0.00 | 40 |
|Silício | 12 | 19 |


### Código

Carrega dados

In [9]:
# composições mínimas e máximas dos componentes
n = [{'c': 0.0, 's': 0.19}, # Composição mínima da Liga 1
    {'c': 0.0, 's': 0.12}]  # Composição mínima da Liga 2

m = [{'c': 0.095, 's': 0.2}, # Composição máxima da Liga 1
     {'c': 0.4, 's': 0.19}]  # Composição máxima da Liga 2

# quantidade desejada da liga
Q = [10, 6]

In [11]:
#                                                                           Não iremos alterar o estoque e a função objetivo.
modelo = Model(sense=MINIMIZE, solver_name=CBC)

# Criação da variável
x = []
x.append(modelo.add_var(var_type=CONTINUOUS, name=f"x_lingotes", lb=0.0))
x.append(modelo.add_var(var_type=CONTINUOUS, name=f"x_grafite", lb=0.0))
x.append(modelo.add_var(var_type=CONTINUOUS, name=f"x_sucata", lb=0.0))

# estoque
modelo += x[0] <= e['l']
modelo += x[1] <= e['g']
modelo += x[1] <= e['s']

# Função objetivo = minimizar custo total
modelo.objective = c['l']*x[0] + c['g']*x[1] + c['s']*x[2] # Minimizar esse custo, dado que lingote é 90$, grafite é 180 e sucate é 25

In [12]:

# Agora, temos que ter o mínimo necessário para a liga 1 + o mínimo necessário para a liga 2.
# O mesmo para o máximo.
# Restrição: quantidade minína de carbono
modelo += a['l']['c']*x[0] + a['g']['c']*x[1] + a['s']['c']*x[2] >= n[0]['c']*10 + n[1]['c']*6

# Restrição: quantidade máxima de carbono
modelo += a['l']['c']*x[0] + a['g']['c']*x[1] + a['s']['c']*x[2] <= m[0]['c']*10 + m[1]['c']*6 # Restrição 1: quantidade máxima de carbono

# Restrição: quantidade miníma de silício
modelo += a['l']['s']*x[0] + a['g']['s']*x[1] + a['s']['s']*x[2] >= n[0]['s']*10 + n[1]['s'] * 6
modelo += a['l']['s']*x[0] + a['g']['s']*x[1] + a['s']['s']*x[2] <= m[0]['s']*10 + m[1]['s'] * 6

# a soma total tem que dar agora Q[0] + Q[1]
modelo += x[0] + x[1] + x[2] == Q[0] + Q[1]

modelo.write("model.lp") # salva modelo em arquivo
with open("model.lp") as f: # lê e exibe conteúdo do arquivo
  print(f.read())

solve(modelo)

\Problem name: 

Minimize
OBJROW: 90 x_lingotes + 180 x_grafite + 25 x_sucata
Subject To
constr(0):  x_lingotes <= 5
constr(1):  x_grafite <= 5
constr(2):  x_grafite <= 12
constr(3):  0.00500 x_lingotes + 0.90000 x_grafite + 0.09000 x_sucata >= -0
constr(4):  0.00500 x_lingotes + 0.90000 x_grafite + 0.09000 x_sucata <= 3.35000
constr(5):  0.14000 x_lingotes + 0.27000 x_sucata >= 2.62000
constr(6):  0.14000 x_lingotes + 0.27000 x_sucata <= 3.14000
constr(7):  x_lingotes + x_grafite + x_sucata = 16
Bounds
End

Status =  OptimizationStatus.OPTIMAL
Solution value  = 1029.26

Solution:
x_lingotes = 5.00
x_grafite = 1.96
x_sucata = 9.04
